In [1]:
#in the name of god

In [2]:
import pandas as pd 
import numpy as np
from IPython.display import clear_output
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import matplotlib.cm as cm
import matplotlib as matplotlib

from fastdtw import fastdtw

def color_map_color(value, cmap_name='coolwarm', vmin=0, vmax=10):
    # norm = plt.Normalize(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap(norm(abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color
cl=['r','g','b','c','m','y','k']
color=dict()
for i,el in enumerate(cl):
    color.update({i:el})
#extract the labels for clutering precision its for after clustering
def time_convertor(x):
    s,h,m=0,0,0
    s=round(x%60,2)
    m=int(x/60)
    h=int(m/60)
    m=m%60
    time=str(h)+':'+str(m)+':'+str(s)
    return pd.to_datetime('1970-01-01 '+time)


# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 5)})

In [3]:
#data_read
def read_data(number):
    #prepare Dataset gyroscope
    dfg=pd.read_csv(f'{number}\\giroscopio_terra.csv')
    dfg['ts']=pd.to_datetime(dfg.apply(lambda x:(round((x.uptimeNanos-dfg.uptimeNanos[0])/1000000)*1000000),axis=1))
    dm=dfg['ts']
    dfg=dfg.drop('uptimeNanos',1).drop('timestamp',1)
    dfg=dfg.set_index('ts').rename(columns={'x': 'g-x','y': 'g-y','z': 'g-z'})
    #prepare Dataset accelarator
    dfa=pd.read_csv(f'{number}\\acelerometro_terra.csv')
    dfa['ts']=dm
    dfa=dfa.drop('uptimeNanos',1).drop('timestamp',1)
    dfa=dfa.set_index('ts').rename(columns={'x': 'a-x','y': 'a-y','z': 'a-z'})
    dfg['g-x']=dfg.apply(lambda x:(x[0]-min(dfg['g-x']))/(max(dfg['g-x'])-min(dfg['g-x'])),axis=1)
    dfg['g-y']=dfg.apply(lambda x:(x[1]-min(dfg['g-y']))/(max(dfg['g-y'])-min(dfg['g-y'])),axis=1)
    dfg['g-z']=dfg.apply(lambda x:(x[2]-min(dfg['g-z']))/(max(dfg['g-z'])-min(dfg['g-z'])),axis=1)
    dfa['a-x']=dfa.apply(lambda x:(x[0]-min(dfa['a-x']))/(max(dfa['a-x'])-min(dfa['a-x'])),axis=1)
    dfa['a-y']=dfa.apply(lambda x:(x[1]-min(dfa['a-y']))/(max(dfa['a-y'])-min(dfa['a-y'])),axis=1)
    dfa['a-z']=dfa.apply(lambda x:(x[2]-min(dfa['a-z']))/(max(dfa['a-z'])-min(dfa['a-z'])),axis=1)
    return pd.concat([dfa, dfg], axis=1, join='outer')
dfRaw=list()
for num in [16,17,20,21]:
    dfRaw.append((num,read_data(num)))
with open("normalized_data.txt", "wb") as fp:
    pickle.dump(dfRaw, fp)

In [3]:
with open("normalized_data.txt", "rb") as fp:
        dfRaw = pickle.load(fp)
        
def get_label(number,df):
    label_event_lenght=dict()
    df=pd.read_csv(f'{number}\\groundTruth.csv')
    df['length']=df.en-df.st
    for event , dft in df.groupby('evento'):
        if (event in label_event_lenght):
            label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
        else:
            label_event_lenght.update({event : list(dft.length)})

    df['st_time']=df.apply(lambda x:time_convertor(x.st) ,axis=1 )
    df['en_time']=df.apply(lambda x:time_convertor(x.en) ,axis=1 )
    df=df.drop('st',1).drop('en',1).drop('length',1)
    evented_label=list()
    for i in range(len(df)):
        evented_label.append((df.iloc[i][0],df.iloc[i][1],df.iloc[i][2]))
    eve=pd.DataFrame(evented_label)
    eve[3]=[f'evented_label_{number}' for x in evented_label]
    return eve
Ev=[get_label(dfRaw[x][0],dfRaw[x][1]) for x in range(len(dfRaw))]
evented_label_All=pd.concat(Ev).reset_index().drop('index',1)


In [4]:
rate=(dfRaw[0][1].index[11]-dfRaw[0][1].index[10]).total_seconds()

print(f"rate is :{rate}")
pd.DataFrame([(x[0],round(((x[2]-x[1]).total_seconds())/rate)) for x in evented_label_All.iloc]).groupby(0).agg({1:"max"})

rate is :0.02


,1
0,
aceleracao_agressiva,245
curva_direita_agressiva,225
curva_esquerda_agressiva,225
evento_nao_agressivo,225
freada_agressiva,185
troca_faixa_direita_agressiva,125
troca_faixa_esquerda_agressiva,120


In [5]:
#supervised
window={226:['curva_direita_agressiva','curva_esquerda_agressiva','aceleracao_agressiva'],
        126:['troca_faixa_direita_agressiva','troca_faixa_esquerda_agressiva'],
        186:['freada_agressiva']
       }

In [6]:
#Dont RUN this part
#we should optimize M
#lengh of random event
import random
random.seed(123)
def TempEvent_genarator(df,L,i):
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        M=round(random.gauss(0.5,0.5)*L/2)
        clear_output(wait=True)
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        print('extraction ',l_max/len(df),' % of ',i,' is complete')
        temp_event.append(df[int(l_min):int(l_max)])
    return temp_event

temp_event=list()
for l,DF in enumerate([x[1] for x in dfRaw]):
    te=list()
    for i,lw in enumerate(window):
        te.append((window[lw],TempEvent_genarator(DF,lw,i+1)))
    temp_event.append(te)

with open("temp_event_new.txt", "wb") as fp:
    pickle.dump(temp_event, fp)

extraction  1.0  % of  3  is complete


In [7]:
with open("temp_event_new.txt", "rb") as fp:
    temp_event = pickle.load(fp)

In [8]:
def has_overlap(A_start, A_end, B_start, B_end):
    latest_start = max(A_start, B_start)
    earliest_end = min(A_end, B_end)
    return latest_start <= earliest_end

def export_labels(tempevent,evented_label):
    EvL=pd.DataFrame(evented_label)
    Labels={
          226:[],
          126:[],
          186:[],
           }
    for i,wind in enumerate(Labels):
        label=['NAG' for x in tempevent[i][1]]
        for name in tempevent[i][0]:
            el=[(x[1],x[2]) for x in EvL.iloc if x[0]==name]
            for j,te in enumerate(tempevent[i][1]):
                    sta =te.index[0]
                    ena =te.index[-1]
                    for stb,enb in el:
                        if has_overlap(sta,ena,stb,enb):
                            label[j]=name
        Labels[wind]=label
    return Labels


Data={
      226:[],
      126:[],
      186:[],
       }
for i,temp in enumerate(temp_event):
    TempEv={
              226:[ x for x in temp_event[i][0][1] ],
              126:[ x for x in temp_event[i][1][1] ],
              186:[ x for x in temp_event[i][2][1] ],
               }
    Label=export_labels(temp_event[i],Ev[i])
    #merge together
    for wind in Data:
        for j in range(len(TempEv[wind])):
            Data[wind].append((Label[wind][j],TempEv[wind][j]))


In [9]:
Data[226][10]

('NAG',
                               a-x       a-y       a-z       g-x       g-y  \
 ts                                                                          
 1970-01-01 00:00:08.609  0.434717  0.495165  0.372532  0.584724  0.660125   
 1970-01-01 00:00:08.629  0.434815  0.443337  0.369497  0.520917  0.743245   
 1970-01-01 00:00:08.648  0.491156  0.511841  0.331410  0.485508  0.750034   
 1970-01-01 00:00:08.668  0.529514  0.576479  0.332659  0.568974  0.723956   
 1970-01-01 00:00:08.688  0.521595  0.473574  0.335476  0.589592  0.688791   
 ...                           ...       ...       ...       ...       ...   
 1970-01-01 00:00:12.948  0.435919  0.445521  0.392083  0.541604  0.684052   
 1970-01-01 00:00:12.968  0.443924  0.452585  0.371189  0.492296  0.726309   
 1970-01-01 00:00:12.987  0.490837  0.511433  0.338475  0.513746  0.762484   
 1970-01-01 00:00:13.007  0.503200  0.550687  0.326413  0.579276  0.666320   
 1970-01-01 00:00:13.026  0.487606  0.501061  0.340951  

In [10]:
#dont run this part
#calculate dist of Data from labdeled event
t1=time.time()

k,n=0,0
num_event=dict(evented_label_All.groupby(0).size())
for lenght in window:
    for name in window[lenght]:
        k=k+len(Data[lenght])*num_event[name]

Data_DS={
      226:[],
      126:[],
      186:[],
       }

for wind in Data:
    for label,event in Data[wind]:
         #---------for each Data-distance from events----------
        Y=list()
        for from_event,dfe in evented_label_All.groupby(0):
            if from_event in window[wind]:
                X=list()
                #____for each event_distance from each label_____
                for el in dfe.iloc:
                    clear_output(wait=True)
                    n=n+1
                    print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
                    if   el[3]=='evented_label_16':
                        X.append(fastdtw(event,dfRaw[0][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_17':
                        X.append(fastdtw(event,dfRaw[1][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_20':
                        X.append(fastdtw(event,dfRaw[2][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_21':
                        X.append(fastdtw(event,dfRaw[3][1][el[1]:el[2]])[0])
                #_________________________________________________
                Y.append((from_event,X))
        Data_DS[wind].append((label,Y))
        #------------------------------------------------------

t2=time.time()
with open("Disstance_new.txt", "wb") as fp:
    pickle.dump(Data_DS, fp)

print(round((t2-t1)/60) ,'min has time for calculaton distant')

disstance calculatoin  100.0  %  is complete
210 min has time for calculaton distant


In [11]:
with open("Disstance_new.txt", "rb") as fp:
    Data_DS = pickle.load(fp)

In [12]:
Data_DS[226][10]

('NAG',
 [('aceleracao_agressiva',
   [191.54451527900127,
    196.38316303046787,
    206.10318235888877,
    190.36035998875744,
    215.45851999560674,
    201.6141655570143,
    195.7536650022861,
    211.49902944277997,
    186.4721907691484,
    192.15442188996943,
    186.2779097408009,
    189.36629647950832]),
  ('curva_direita_agressiva',
   [139.1863155699013,
    134.0729931726308,
    138.01092977305652,
    138.33203932961524,
    132.61000791284755,
    195.92303721517737,
    184.09107164297535,
    188.88327881457244,
    185.7430090706699,
    200.07598574250963,
    200.84831426313477]),
  ('curva_esquerda_agressiva',
   [141.5015798031517,
    146.40583349086597,
    141.23016066764876,
    135.5944426447342,
    135.0949140259224,
    147.73807836029138,
    147.57293721169253,
    189.1883871648448,
    182.55261910752938,
    150.69019361607639,
    143.56230971182543])])

In [13]:
#prepare dataset for labeling
TrainData=dict()
for win in Data_DS:
    x=Data_DS[win]
    dumy_list=list()
    for events in x:
        dumy_dict=dict()
        dumy_dict.update({'label':events[0]})
        for name , event in events[1]:
            for tag,number in enumerate(event):
                dumy_dict.update({name+str(tag):number})
        dumy_list.append(dumy_dict)
    TrainData.update({win:pd.DataFrame(dumy_list)})

In [14]:
TrainData[226].head(10)

,label,aceleracao_agressiva0,aceleracao_agressiva1,aceleracao_agressiva2,aceleracao_agressiva3,aceleracao_agressiva4,aceleracao_agressiva5,aceleracao_agressiva6,aceleracao_agressiva7,aceleracao_agressiva8,...,curva_esquerda_agressiva1,curva_esquerda_agressiva2,curva_esquerda_agressiva3,curva_esquerda_agressiva4,curva_esquerda_agressiva5,curva_esquerda_agressiva6,curva_esquerda_agressiva7,curva_esquerda_agressiva8,curva_esquerda_agressiva9,curva_esquerda_agressiva10
0,NAG,152.197999,160.497296,186.592491,165.302999,210.106082,196.125649,173.099501,214.405838,162.520954,...,134.069696,130.659010,127.619507,125.083264,136.117751,119.478497,169.938228,169.936294,131.389872,117.843499
1,NAG,174.132673,186.612654,195.795481,181.111607,209.379603,195.172057,196.726296,214.388954,186.094920,...,137.802186,130.092994,128.639639,125.737518,147.348257,135.079695,184.270782,177.154381,150.860153,138.053369
2,NAG,180.011335,190.334389,198.662247,178.965790,207.663424,195.148659,195.247498,212.591710,184.101841,...,139.716997,130.120482,129.099205,127.764363,148.583594,141.254033,183.945023,175.007803,149.724870,142.797234
3,NAG,187.901446,197.688761,204.792589,183.060550,209.385808,194.333629,191.702616,208.909757,181.404671,...,146.508009,136.767687,132.158555,133.325194,145.808301,148.953552,188.733119,176.193748,145.361263,143.916225
4,NAG,189.349957,195.899470,205.721027,188.775423,215.416464,202.042344,189.722289,206.523534,180.485032,...,141.166271,141.219495,131.388284,131.289693,144.321050,144.376773,186.616092,179.565794,144.596604,140.896358
5,NAG,187.384897,194.009647,207.124083,190.775561,215.979745,202.112077,191.874606,208.281975,182.014288,...,141.376081,137.099474,134.377320,132.532075,147.059851,141.723150,187.675786,181.846478,147.281289,139.569838
6,NAG,188.813968,195.058204,206.458820,190.177843,216.031373,202.839420,192.046547,208.498959,181.821516,...,140.771136,137.749359,133.784181,133.603994,147.262097,141.119745,187.429157,181.148182,148.028778,139.236481
7,NAG,187.994730,194.484463,205.107036,190.266882,215.539398,201.263330,194.631856,211.023260,185.166637,...,145.201152,139.747416,138.259912,138.498218,147.851698,143.489794,190.351302,183.353196,149.954194,140.850625
8,NAG,189.131907,194.796002,204.503946,188.761881,213.573636,200.159248,195.048884,210.132569,186.618618,...,145.030567,138.186932,135.035726,134.092135,148.265918,146.585596,188.933583,180.947428,150.665747,141.113163
9,NAG,190.102388,196.620819,206.544598,189.718784,214.251714,201.285006,195.607015,211.565676,187.697392,...,146.620132,139.287957,135.881675,137.095633,149.328110,146.931399,189.254757,181.720578,152.030709,144.604059


In [15]:
#normalize Train Data
#Balance it 

In [21]:
#dont run this part
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report
t1=time.time()
n,k=0,12*50

Result={
      226:[],
      126:[],
      186:[],
       }

for wid in Result:
    data=TrainData[wid].drop('label',1)
    label=TrainData[wid].label
    #
    for Max_depth in range(1,13):
        estimator = Id3Estimator(max_depth=Max_depth, min_samples_split=1, prune=True,
                        gain_ratio=True, min_entropy_decrease=0, is_repeating=True)
        #run it in itaraion for the best trees
        NUMBER_OF_TEST=50
        temp=list()
        for NT in range(NUMBER_OF_TEST):
            clear_output(wait=True)
            n=n+1
            print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
            estimator.fit(data,label , check_input=True)
            ACC=accuracy_score(y_true=label,y_pred=estimator.predict(data))
            temp.append((ACC,deepcopy(estimator)))
        temp_index=[(x[0],i) for i,x in enumerate(temp)]
        temp_index.sort()
        Acc=temp[temp_index[-1][1]][0]
        estimator=temp[temp_index[-1][1]][1]
        Result[wid].append((Acc,deepcopy(estimator)))

with open("Model_new_1.txt", "wb") as fp:
    pickle.dump(Result, fp)
t2=time.time()
print(round((t2-t1)/60) ,'min has time for calculaton Model')

disstance calculatoin  300.0  %  is complete
356 min has time for calculaton Model


In [17]:
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

with open("Model_new_1.txt", "rb") as fp:
    Result = pickle.load(fp)

In [25]:
Max_depth=7
wid=126

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9927609089081038
                                precision    recall  f1-score   support

                           NAG       0.99      1.00      1.00      4896
 troca_faixa_direita_agressiva       0.96      0.59      0.73        46
troca_faixa_esquerda_agressiva       0.78      0.68      0.72        31

                      accuracy                           0.99      4973
                     macro avg       0.91      0.75      0.82      4973
                  weighted avg       0.99      0.99      0.99      4973



In [26]:
Max_depth=7
wid=186

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9888721804511278
                  precision    recall  f1-score   support

             NAG       0.99      1.00      0.99      3234
freada_agressiva       0.90      0.67      0.77        91

        accuracy                           0.99      3325
       macro avg       0.94      0.83      0.88      3325
    weighted avg       0.99      0.99      0.99      3325



In [27]:
Max_depth=7
wid=226

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9534528805799313
                          precision    recall  f1-score   support

                     NAG       0.95      1.00      0.97      2379
    aceleracao_agressiva       0.96      0.27      0.42        81
 curva_direita_agressiva       1.00      0.52      0.68        62
curva_esquerda_agressiva       0.99      0.69      0.81        99

                accuracy                           0.95      2621
               macro avg       0.97      0.62      0.72      2621
            weighted avg       0.95      0.95      0.94      2621



In [ ]:
#overfit model!
Overfit_model={
      226:[],
      126:[],
      186:[],
       }

for wid in Overfit_model:
    data=TrainData[wid].drop('label',1)
    label=TrainData[wid].label
    #
    estimator = Id3Estimator(max_depth=100, min_samples_split=1, prune=True,
                    gain_ratio=True, min_entropy_decrease=0, is_repeating=True)
    #run it in itaraion for the best trees
    NUMBER_OF_TEST=10
    temp=list()
    for NT in range(NUMBER_OF_TEST):
        estimator.fit(data,label , check_input=True)
        ACC=accuracy_score(y_true=label,y_pred=estimator.predict(data))
        temp.append((ACC,deepcopy(estimator)))
    temp_index=[(x[0],i) for i,x in enumerate(temp)]
    temp_index.sort()
    Acc=temp[temp_index[-1][1]][0]
    estimator=temp[temp_index[-1][1]][1]
    Overfit_model[wid]=(Acc,deepcopy(estimator))

with open("TrainData_test_l_1.txt", "wb") as fp:
    pickle.dump((Overfit_model), fp)



In [ ]:
wid=226
data=TrainData[wid].drop('label',1)
label=TrainData[wid].label
print(classification_report(label, Overfit_model[wid][1].predict(data)))
#its completely ovefit

In [ ]:
for in wid window:
    estimator=Result[wid][5][1]
    x={x for x in estimator.predict(Test_new[wid])}
    print(f" {wid} is : {x}")


In [ ]:
import re
x=data.columns[30]
rep=re.compile(r'(\w+)(\d+)')
mo=rep.findall(x)[0]
print(mo[])


In [ ]:
#plot charts for find the optimom
Y1=[x[0]for x in Result[226]]
Y2=[x[0]for x in Result[186]]
Y3=[x[0]for x in Result[126]]

X=range(1,11)

fig, ax = plt.subplots()

ax.plot(X,Y1, linestyle='-', linewidth=2,c='r')
ax.plot(X,Y2, linestyle='-', linewidth=2,c='g')
ax.plot(X,Y3, linestyle='-', linewidth=2,c='b')

ax.legend();
plt.show() 